In [10]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import joblib

In [7]:
df = pd.read_csv('/content/drive/MyDrive/salary_sport.csv')
df.drop(columns= 'Unnamed: 0')

,id,пол,возраст,наличие_медали,зарплата_тыс
0,1,0,25,4,128
1,2,0,32,7,157
2,3,0,33,4,162
3,4,0,39,10,201
4,5,1,21,6,138
...,...,...,...,...,...
795,796,0,39,12,229
796,797,0,27,2,96
797,798,0,20,8,147
798,799,1,20,8,146


In [9]:
X = df.drop(columns=['зарплата_тыс'])
y = df['зарплата_тыс']

# Разделение на обучающий и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Размер обучающего набора данных (X_train):", X_train.shape)
print("Размер тестового набора данных (X_test):", X_test.shape)
print("Размер обучающего набора данных (y_train):", y_train.shape)
print("Размер тестового набора данных (y_test):", y_test.shape)


Размер обучающего набора данных (X_train): (640, 5)
Размер тестового набора данных (X_test): (160, 5)
Размер обучающего набора данных (y_train): (640,)
Размер тестового набора данных (y_test): (160,)


In [11]:
# Инициализация и обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [13]:
# Прогнозирование на тестовом наборе данных
y_pred = model.predict(X_test)

# Вычисление средней абсолютной ошибки (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("Средняя абсолютная ошибка (MAE):", mae)

Средняя абсолютная ошибка (MAE): 19.865217515740785


In [14]:
# Сохранение модели
joblib.dump(model, 'linear_regression_model.pkl')

['linear_regression_model.pkl']